In [2]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Load the cleaned datasets
try:
    benin_df = pd.read_csv('../data/benin-solar-farm.csv')
    sierra_leone_df = pd.read_csv('../data/sierraleone-solar-farm.csv')
    togo_df = pd.read_csv('../data/togo-solar-farm.csv')
except FileNotFoundError:
    print("Make sure your cleaned CSV files are in the 'data/' directory.")
    # As a fallback for demonstration if files are not found, create empty dataframes
    benin_df = pd.DataFrame()
    sierra_leone_df = pd.DataFrame()
    togo_df = pd.DataFrame()


# Add a 'Country' column to each dataframe before combining them
benin_df['Country'] = 'Benin'
sierra_leone_df['Country'] = 'Sierra Leone'
togo_df['Country'] = 'Togo'

# Combine the three dataframes into a single one
combined_df = pd.concat([benin_df, sierra_leone_df, togo_df], ignore_index=True)

# Convert Timestamp to datetime object, just in case
combined_df['Timestamp'] = pd.to_datetime(combined_df['Timestamp'])

print("Data loaded and combined successfully!")
print("Combined DataFrame shape:", combined_df.shape)
combined_df.head()

Data loaded and combined successfully!
Combined DataFrame shape: (26280, 11)


,Timestamp,GHI,DNI,DHI,ModA,ModB,Tamb,RH,WS,Cleaning,Country
0,2022-01-01 00:00:00,100.05,87.27,12.78,95.05,94.05,21.80,60.09,1.16,0,Benin
1,2022-01-01 01:00:00,48.60,37.85,10.75,46.17,45.68,24.88,60.63,2.92,0,Benin
2,2022-01-01 02:00:00,52.49,37.51,14.98,49.86,49.34,23.13,55.60,1.87,0,Benin
3,2022-01-01 03:00:00,46.83,34.94,11.89,44.49,44.02,25.23,52.43,3.21,0,Benin
4,2022-01-01 04:00:00,34.80,25.92,8.88,33.06,32.71,25.11,48.89,2.01,0,Benin
